https://arxiv.org/pdf/2110.01368.pdf

https://arxiv.org/pdf/1911.03380.pdf

https://deliverypdf.ssrn.com/delivery.php?ID=773124118097097119026017080102079065057081049043000029071005117069008065110098087085056017039010001111005112084088095103006007043087058092007068069090091015080092022071021048008071122094090110120127079071113126114121118104123004007070096119111104095125&EXT=pdf&INDEX=TRUE

Let $\Sigma_t$ denote portfolio of the instruments under our managment
$$\Sigma_t = n_i(t) \cdot I_i(t),$$
where $n_i(t)$ -- position in the instrument $i$, $I_i(t)$ -- its present value

$$\dot\Sigma(t) = \dot n_i I_i + n_i \dot I_i$$

So we must implement Instruments' dynamics and portolio strategy to control $n_i$ 

In [1]:
import numpy as np
import pandas as pd
from LPtools.utils import Adapter, Runner
from LPtools.clients import Client
from LPtools.instruments import UniPool, Bond, Portfolio, Position, Cash, Perpetual
from LPtools.instruments import Holder, PerpetHedger

In [2]:
zcurve = pd.read_csv('zcyc_rates.csv', index_col=0, parse_dates=True, infer_datetime_format=True, na_filter=False)

adapter = Adapter(pool_id="0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
                  symbol="ETHUSDT",
                  zcurve=zcurve)

#response = adapter(300)

In [3]:
#response.to_csv('saved_response.csv')

In [4]:
response = pd.read_csv('saved_response.csv', index_col=0, infer_datetime_format=True)
init_state = response.iloc[0]

1. Hodl

In [5]:
perpetual = Position(tag='perpet', instrument=Perpetual())
bond = Position(tag='bond', instrument=Bond(init_state))

positions = [perpetual, bond]
balancer = Holder(positions, initial_amounts=[1 / init_state.token0Price, -1])
portfolio_hodl = Portfolio(balancer, positions=positions)

2. Passive pool

In [6]:

pool = Position(tag='pool', instrument=UniPool(init_state , price_lower=2000, price_upper=5000, fees=.005))
bond = Position(tag='bond', instrument=Bond(init_state))


positions = [pool, bond]
balancer = Holder(positions, initial_amounts=[1,-1])
portfolio_pooler = Portfolio(balancer, positions=positions)

3. Hedging

In [38]:
pool = Position(tag='pool', instrument=UniPool(init_state , price_lower=2000, price_upper=5000, fees=.005))
bond = Position(tag='bond', instrument=Bond(init_state))
perpetual = Position(tag='perpet', instrument=Perpetual())

positions = [pool, perpetual, bond]

delta =  pool.instrument.L / init_state.token0Price**.5
balancer = PerpetHedger(
    positions,
    initial_amounts=[1, - delta, delta * init_state.token0Price - 1 ],
    rebalancing_interval= 5
)
portfolio_perpet = Portfolio(balancer, positions)

In [32]:
bond.instrument.value(init_state)

1.029570636022149

In [33]:
pool.instrument.value(init_state)

1.0

In [34]:
perpetual.instrument.value(init_state)

3496.203173435439

In [40]:
portfolio_perpet.step(init_state)
list(map(lambda p: p.value(init_state), [pool, bond, perpetual]))

[1.0, -0.5000697222222222, 0.5]

In [25]:
runner = Runner(response,[portfolio_hodl, portfolio_pooler, portfolio_perpet])
runner.run()

In [26]:
runner.summary(2)

pool_last_val 1.1513300420579644
perpet_last_val 0.612231038932429
bond_last_val 7199.176249873161
total_value 7140.790229963204
discounted_value 3625.1291242235366


,pool,perpet,bond,payments,transaction_costs
0,1.000013,3496.292879,1.000000,0.000000e+00,-6.997406e+01
1,0.998211,3483.701459,1.000139,1.406735e-06,0.000000e+00
2,1.054052,3884.370431,1.000279,-8.511378e-05,0.000000e+00
3,1.057450,3909.453752,1.000418,9.635210e-07,0.000000e+00
4,1.065387,3968.366567,1.000558,-1.234656e-05,0.000000e+00
...,...,...,...,...,...
204,1.075175,4041.612472,1.028853,4.048076e-05,0.000000e+00
205,1.082566,4097.371402,1.028997,-2.200563e-06,-5.770671e-10
206,1.108335,4294.761309,1.029140,-7.882936e-06,0.000000e+00
207,1.127702,4446.166582,1.029284,-5.927545e-06,0.000000e+00
